In [8]:
import joblib
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.weightstats import DescrStatsW
import numpy as np
import pandas as pd
from ALSampler import summarize
from itertools import chain
from pathlib import Path

In [40]:
m_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_crf_scores_sentenceSampling_500bootstrap.joblib')
mfull_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_crf_scores_sentenceSampling_full500bootstrap.joblib')
r_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_random_scores_sentenceSampling_500bootstrap.joblib')
vmax_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_VBmax_scores_sentenceSampling_500bootstrap.joblib')
vmin_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_VBmin_scores_sentenceSampling_500bootstrap.joblib')

In [29]:
name1='m'
name2='v'
measures=['precision','recall','f1'] 
cols=[b+"_"+a for b in measures for a in [name1, name2,'pv'] ]
results={col:[] for col in cols}

In [30]:
results

{'precision_m': [],
 'precision_v': [],
 'precision_pv': [],
 'recall_m': [],
 'recall_v': [],
 'recall_pv': [],
 'f1_m': [],
 'f1_v': [],
 'f1_pv': []}

In [31]:
def compute_stats(scores1, scores2, name1, name2, runs=3, rounds=10):
    measures=['precision','recall','f1'] 
    cols=[b+"_"+a for b in measures for a in [name1, name2,'pv'] ]
    results={col:[] for col in cols}
    for ri in range(rounds):
        single_round_scores={name1:{}, name2:{}}
        for m in measures:             
            # print('s1',len(scores1), len(scores1[0]), len(scores1[0][0][m]))
            # print('s2',len(scores2), len(scores2[0]), len(scores2[0][0][m]))
            single_round_scores[name1][m]=list(chain.from_iterable([scores1[u][ri][m] for u in range(runs)]))
            single_round_scores[name2][m]=list(chain.from_iterable([scores2[u][ri][m] for u in range(runs)]))
            results[m+'_'+name1].append(np.mean(single_round_scores[name1][m]))
            results[m+'_'+name2].append(np.mean(single_round_scores[name2][m]))
            ttres=ttest_ind(single_round_scores[name1][m], single_round_scores[name2][m], usevar='unequal')
            # ttres=DescrStatsW.ttest_mean(np.array(single_round_scores[name1][m])-np.array(single_round_scores[name2][m]))
            results[m+'_pv'].append(ttres[1])
    return pd.DataFrame(results)

In [32]:
list(range(3))

[0, 1, 2]

## model vs full uncertainty model

In [41]:
results=compute_stats(m_scores, mfull_scores, 'model', 'fullmodel')

In [42]:
results

,precision_model,precision_fullmodel,precision_pv,recall_model,recall_fullmodel,recall_pv,f1_model,f1_fullmodel,f1_pv
0,0.949306,0.949306,1.000000e+00,0.491953,0.491953,1.0,0.647992,0.647992,1.0
1,0.877609,0.927830,0.000000e+00,0.703067,0.624118,0.0,0.780154,0.746216,0.0
2,0.911012,0.933021,0.000000e+00,0.734339,0.650199,0.0,0.813172,0.766288,0.0
3,0.908927,0.925766,0.000000e+00,0.768613,0.687422,0.0,0.832889,0.788941,0.0
4,0.919421,0.933473,3.244188e-234,0.780600,0.709953,0.0,0.844302,0.806199,0.0
5,0.919690,0.932374,0.000000e+00,0.794186,0.733903,0.0,0.852312,0.821225,0.0
6,0.922395,0.932150,0.000000e+00,0.804296,0.751462,0.0,0.859274,0.832064,0.0
7,0.922415,0.933726,0.000000e+00,0.813884,0.763778,0.0,0.864750,0.840233,0.0
8,0.923908,0.934448,0.000000e+00,0.818746,0.780493,0.0,0.868146,0.850548,0.0


### model vs vmin

In [33]:
results=compute_stats(m_scores, vmin_scores, 'model', 'vmin')

In [34]:
results

,precision_model,precision_vmin,precision_pv,recall_model,recall_vmin,recall_pv,f1_model,f1_vmin,f1_pv
0,0.949306,0.949306,1.0,0.491953,0.491953,1.0,0.647992,0.647992,1.0
1,0.877609,0.920320,0.0,0.703067,0.613026,0.0,0.780154,0.735803,0.0
2,0.911012,0.929878,0.0,0.734339,0.618964,0.0,0.813172,0.743152,0.0
3,0.908927,0.936932,0.0,0.768613,0.652374,0.0,0.832889,0.769152,0.0
4,0.919421,0.937744,0.0,0.780600,0.686759,0.0,0.844302,0.792835,0.0
5,0.919690,0.940245,0.0,0.794186,0.697421,0.0,0.852312,0.800820,0.0
6,0.922395,0.938327,0.0,0.804296,0.713505,0.0,0.859274,0.810604,0.0
7,0.922415,0.937745,0.0,0.813884,0.723315,0.0,0.864750,0.816683,0.0
8,0.923908,0.934911,0.0,0.818746,0.737418,0.0,0.868146,0.824482,0.0


### model vs vmax

In [35]:
results=compute_stats(m_scores, vmax_scores, 'model', 'vmax')
results

,precision_model,precision_vmax,precision_pv,recall_model,recall_vmax,recall_pv,f1_model,f1_vmax,f1_pv
0,0.949306,0.956706,4.578798e-175,0.491953,0.457318,0.0,0.647992,0.618810,0.0
1,0.877609,0.944391,0.000000e+00,0.703067,0.588852,0.0,0.780154,0.725386,0.0
2,0.911012,0.946244,0.000000e+00,0.734339,0.608812,0.0,0.813172,0.740907,0.0
3,0.908927,0.943332,0.000000e+00,0.768613,0.648687,0.0,0.832889,0.768735,0.0
4,0.919421,0.944108,0.000000e+00,0.780600,0.681896,0.0,0.844302,0.791852,0.0
5,0.919690,0.944847,0.000000e+00,0.794186,0.692200,0.0,0.852312,0.799021,0.0
6,0.922395,0.943056,0.000000e+00,0.804296,0.706461,0.0,0.859274,0.807785,0.0
7,0.922415,0.943175,0.000000e+00,0.813884,0.716988,0.0,0.864750,0.814667,0.0
8,0.923908,0.938661,0.000000e+00,0.818746,0.732880,0.0,0.868146,0.823099,0.0


### model vs random

In [36]:
results=compute_stats(m_scores, r_scores, 'model', 'rand')
results

,precision_model,precision_rand,precision_pv,recall_model,recall_rand,recall_pv,f1_model,f1_rand,f1_pv
0,0.949306,0.949306,1.0,0.491953,0.491953,1.0,0.647992,0.647992,1.0
1,0.877609,0.942603,0.0,0.703067,0.578504,0.0,0.780154,0.716829,0.0
2,0.911012,0.942485,0.0,0.734339,0.608940,0.0,0.813172,0.739758,0.0
3,0.908927,0.940958,0.0,0.768613,0.641706,0.0,0.832889,0.762902,0.0
4,0.919421,0.943004,0.0,0.780600,0.660618,0.0,0.844302,0.776861,0.0
5,0.919690,0.943946,0.0,0.794186,0.677866,0.0,0.852312,0.789009,0.0
6,0.922395,0.944182,0.0,0.804296,0.692370,0.0,0.859274,0.798858,0.0
7,0.922415,0.943481,0.0,0.813884,0.705308,0.0,0.864750,0.807185,0.0
8,0.923908,0.942843,0.0,0.818746,0.713214,0.0,0.868146,0.812088,0.0


### vmin vs random

In [37]:
results=compute_stats(vmin_scores, r_scores,  'vmin', 'rand')
results

,precision_vmin,precision_rand,precision_pv,recall_vmin,recall_rand,recall_pv,f1_vmin,f1_rand,f1_pv
0,0.949306,0.949306,1.000000e+00,0.491953,0.491953,1.000000e+00,0.647992,0.647992,1.000000e+00
1,0.920320,0.942603,0.000000e+00,0.613026,0.578504,0.000000e+00,0.735803,0.716829,0.000000e+00
2,0.929878,0.942485,0.000000e+00,0.618964,0.608940,1.721133e-149,0.743152,0.739758,2.738544e-51
3,0.936932,0.940958,1.371115e-57,0.652374,0.641706,6.248075e-153,0.769152,0.762902,1.150141e-149
4,0.937744,0.943004,1.450982e-101,0.686759,0.660618,0.000000e+00,0.792835,0.776861,0.000000e+00
5,0.940245,0.943946,3.161930e-97,0.697421,0.677866,0.000000e+00,0.800820,0.789009,0.000000e+00
6,0.938327,0.944182,3.374004e-287,0.713505,0.692370,0.000000e+00,0.810604,0.798858,0.000000e+00
7,0.937745,0.943481,0.000000e+00,0.723315,0.705308,0.000000e+00,0.816683,0.807185,0.000000e+00
8,0.934911,0.942843,0.000000e+00,0.737418,0.713214,0.000000e+00,0.824482,0.812088,0.000000e+00


### vmax vs random

In [38]:
results=compute_stats(vmax_scores,r_scores,  'vmax', 'rand')
results

,precision_vmax,precision_rand,precision_pv,recall_vmax,recall_rand,recall_pv,f1_vmax,f1_rand,f1_pv
0,0.956706,0.949306,4.578798e-175,0.457318,0.491953,0.000000e+00,0.618810,0.647992,0.000000e+00
1,0.944391,0.942603,4.149777e-47,0.588852,0.578504,1.348110e-117,0.725386,0.716829,1.868662e-149
2,0.946244,0.942485,1.628184e-127,0.608812,0.608940,6.957237e-01,0.740907,0.739758,8.842923e-08
3,0.943332,0.940958,1.797972e-25,0.648687,0.641706,6.167580e-81,0.768735,0.762902,5.176071e-153
4,0.944108,0.943004,2.923215e-07,0.681896,0.660618,0.000000e+00,0.791852,0.776861,0.000000e+00
5,0.944847,0.943946,4.658720e-09,0.692200,0.677866,0.000000e+00,0.799021,0.789009,0.000000e+00
6,0.943056,0.944182,1.524200e-17,0.706461,0.692370,0.000000e+00,0.807785,0.798858,0.000000e+00
7,0.943175,0.943481,5.412340e-03,0.716988,0.705308,0.000000e+00,0.814667,0.807185,0.000000e+00
8,0.938661,0.942843,6.016853e-280,0.732880,0.713214,0.000000e+00,0.823099,0.812088,0.000000e+00


#### vmax vs vmin

In [39]:
results=compute_stats(vmin_scores, vmax_scores, 'vmin', 'vmax')
results

,precision_vmin,precision_vmax,precision_pv,recall_vmin,recall_vmax,recall_pv,f1_vmin,f1_vmax,f1_pv
0,0.949306,0.956706,4.578798e-175,0.491953,0.457318,0.000000e+00,0.647992,0.618810,0.000000e+00
1,0.920320,0.944391,0.000000e+00,0.613026,0.588852,0.000000e+00,0.735803,0.725386,0.000000e+00
2,0.929878,0.946244,0.000000e+00,0.618964,0.608812,5.938992e-307,0.743152,0.740907,3.712960e-49
3,0.936932,0.943332,0.000000e+00,0.652374,0.648687,9.519166e-61,0.769152,0.768735,8.549998e-03
4,0.937744,0.944108,0.000000e+00,0.686759,0.681896,4.336108e-100,0.792835,0.791852,4.919274e-11
5,0.940245,0.944847,0.000000e+00,0.697421,0.692200,1.204003e-198,0.800820,0.799021,2.624833e-53
6,0.938327,0.943056,0.000000e+00,0.713505,0.706461,2.952327e-291,0.810604,0.807785,7.306207e-119
7,0.937745,0.943175,0.000000e+00,0.723315,0.716988,1.167818e-306,0.816683,0.814667,1.238516e-75
8,0.934911,0.938661,0.000000e+00,0.737418,0.732880,4.027839e-101,0.824482,0.823099,2.730139e-27
